# Data는 김도휘 형제님과 김명찬 형제님이 만들어주신 보편지향 기도 데이터를 사용하였습니다. 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

## CSV 에서 기도문 읽어오기
def read_data(path_to_file):
    df = pd.read_csv(path_to_file, dtype=str)
    return df

df = read_data('../../data/pray456_v3.csv')

In [2]:
df.to_csv('../../data/pray456_v3withid.csv')

In [3]:
X = list(df['content'])
y = list(df['label'])
print(len(X))
print(len(y))

774
774


In [4]:
X[0]

'주님, 대림시기를 맞는 교회가 회개와 화해의 생활을 하며 저희에게  오실 아기 예수님을 기쁜 마음으로 맞이할 수 있도록 도와주소서.'

## y data encoding

In [5]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

print(type(y[0]), y[:5])

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
print(integer_encoded[:5])
y= integer_encoded
print(y[:5])

<class 'str'> ['1', '2', '3', '4', '1']
[0 1 2 3 0]
[0 1 2 3 0]


## Split to train/test

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1212)
print(len(x_train), len(x_test), len(y_train), len(y_test))
print(y_train[:5])

619 155 619 155
[3 2 3 1 0]


In [7]:
len(x_train), len(y_train), len(x_test), len(y_test)

(619, 619, 155, 155)

In [8]:
x_train[0]

'주님, 정신적 육체적으로 고통받는 형제들을 위하여 기도하오니  주님께서 몸소 그들을 위로하여 주시고  저희가 그들의 어려움을 함께 나누며 살아갈 수 있도록 도와주소서.'

## 토큰 인덱싱 (token2idx)

In [9]:
from collections import defaultdict

# 단어에 대한 idx 부여
def convert_token_to_idx(token_ls):
    for tokens in token_ls:
        yield [token2idx[token] for token in tokens.split(' ')]
    return

In [10]:
token2idx = defaultdict(lambda : len(token2idx)) # token과 index를 매칭시켜주는 딕셔너리
pad = token2idx['<PAD>']  # pytorch Variable로 변환하기 위해, 문장의 길이를 맞춰주기 위한 padding 

x_train = list(convert_token_to_idx(x_train))
x_test = list(convert_token_to_idx(x_test))

idx2token = {val : key for key,val in token2idx.items()}

#### 인덱싱 결과 확인 

In [11]:
x_train[0]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 8,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22]

#### 원본 텍스트로 변환 확인 

In [12]:
[idx2token[x] for x in x_train[0]]

['주님,',
 '정신적',
 '육체적으로',
 '고통받는',
 '형제들을',
 '위하여',
 '기도하오니',
 '',
 '주님께서',
 '몸소',
 '그들을',
 '위로하여',
 '주시고',
 '',
 '저희가',
 '그들의',
 '어려움을',
 '함께',
 '나누며',
 '살아갈',
 '수',
 '있도록',
 '도와주소서.']

## Add Padding

In [13]:
# Pytorch Variable로 변환하기 위해서는 모든 data의 길이(length)가 동일하여야 한다.
# 영화 리뷰는 길이가 제각각이므로, 길이를 맞춰주는 작업을 수행
# 짧은 문장에는 padding(공간을 채우기 위해 사용하는 더미)을 추가하고,
# 긴 문장은 짤라서 줄인다.

In [14]:
# Sequence Length를 맞추기 위한 padding
def add_padding(token_ls, max_len):
    for i, tokens in enumerate(token_ls):
        n_token = len(tokens)
        
        # 길이가 짧으면 padding을 추가
        if n_token < max_len:
            token_ls[i] += [pad] * (max_len - n_token) # 부족한 만큼 padding을 추가함
        
        # 길이가 길면, max_len에서 짜름
        elif n_token > max_len:
            token_ls[i] = tokens[:max_len]
    return token_ls

In [15]:
max_len = 30
x_train = add_padding(x_train, max_len)
x_test = add_padding(x_test, max_len)

#### Padding 결과 확인

In [16]:
' '.join([idx2token[x] for x in x_train[0]])

'주님, 정신적 육체적으로 고통받는 형제들을 위하여 기도하오니  주님께서 몸소 그들을 위로하여 주시고  저희가 그들의 어려움을 함께 나누며 살아갈 수 있도록 도와주소서. <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'

## Pytorch 모델 학습을 위해 Data의 type을 Variable 로 변환

In [17]:
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.nn.functional as F

In [18]:
# torch Variable로 변환
def convert_to_long_variable(w2i_ls):
    return Variable(torch.LongTensor(w2i_ls))

In [19]:
x_train = convert_to_long_variable(x_train)
x_test = convert_to_long_variable(x_test)

y_train = convert_to_long_variable(y_train)
y_test = convert_to_long_variable(y_test)

In [20]:
x_train[0]

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  8, 14, 15, 16, 17,
        18, 19, 20, 21, 22,  0,  0,  0,  0,  0,  0,  0])

# CBOW with Pytorch

In [457]:
class CBOW(nn.Module):
    def __init__(self, n_words, embed_size, pad_index, hid_size, dropout, n_class):
        super(CBOW, self).__init__()
        
        self.n_words = n_words   # 고유한 토큰의 갯수
        self.embed_size = embed_size   # 임베딩 차원의 크기
        self.pad_index = pad_index     # 문장에 포함된 padding_token, embedding 과정에서 제외시킴
        self.embed = nn.Embedding(n_words, embed_size, padding_idx=pad_index) # non-static embedding with Pytorch
        
        self.hid_size = hid_size    # Fully-Connet layer의 히든 레이어의 갯수
        self.dropout = dropout   # 드롭아웃 비율
        self.n_class = n_class   # 카테고리의 갯수

#         pre-train된 embedding을 사용하고 싶다면,
#         self.embed.weight = pre_trained_weight_matrix
#         self.embed.weight.requires_grad = False  # embedding weight 고정 : static
        
        self.lin = nn.Sequential(
            nn.Linear(embed_size, hid_size), nn.ReLU(),
            nn.Linear(hid_size, hid_size), nn.ReLU(),
            nn.Linear(hid_size, hid_size), nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hid_size, n_class)
        )

    def forward(self, x):
        x_embeded = self.embed(x)  # batch_size x sequence_length x embed_size

        # 모든 단어의 embedding vector를 모두 더하여 sentence를 모델링한다.
        x_cbow = x_embeded.sum(dim=1) # batch_size x 1 x embeded_size
        x_cbow = x_cbow.squeeze(1)    # fully-connet를 위해, 1번째 차원을 축소

        logit = self.lin(x_cbow)
        return logit

In [464]:
params = {
    'n_words' : len(token2idx),     # 고유한 토큰의 갯수
    'embed_size' : 500,                 # embedding 차원의 크기
    'pad_index' : token2idx['<PAD>'],  # embedding 과정에서 제외시킬, padding token
    'hid_size' : 1000,                   # 히든 레이어 갯수
    'dropout' : 0.5,                   # 드롭아웃 비율
    'n_class' : 4,                     # 카테고리 갯수 (긍/부정)
}

In [465]:
model = CBOW(**params)

In [466]:
model

CBOW(
  (embed): Embedding(3334, 500, padding_idx=0)
  (lin): Sequential(
    (0): Linear(in_features=500, out_features=1000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1000, out_features=1000, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1000, out_features=1000, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.5, inplace=False)
    (7): Linear(in_features=1000, out_features=4, bias=True)
  )
)

# Train

In [467]:
import random
import numpy as np

In [468]:
def adjust_learning_rate(optimizer, epoch, init_lr=0.001, lr_decay_epoch=10):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    lr = init_lr * (0.87**(epoch // lr_decay_epoch))

    if epoch % lr_decay_epoch == 0:
        print('LR is set to %s'%(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer

In [ ]:
epochs = 60
lr = 0.0001
batch_size = 32

train_idx = np.arange(x_train.size(0))
test_idx = np.arange(x_test.size(0))
optimizer = torch.optim.Adam(model.parameters(),lr)  # Adam Optimizer 사용
criterion = nn.CrossEntropyLoss(reduction='sum')     # model이 logit을 반환하므로, 크로스-엔트로피-Loss를 사용,
                                                     # 크로스-엔트로피-Loss는 Log_softmax + NLL_loss 
for epoch in range(1, epochs+1):
    model.train()
    
    # input 데이터 순서 섞기
    random.shuffle(train_idx)
    x_train = x_train[train_idx]
    y_train = y_train[train_idx]
    train_loss = 0

    for start_idx, end_idx in zip(range(0, x_train.size(0), batch_size),
                                  range(batch_size, x_train.size(0)+1, batch_size)):
        x_batch = x_train[start_idx : end_idx]
        y_batch = y_train[start_idx : end_idx].long()
        
        scores = model(x_batch)
        predict = F.softmax(scores, dim=1).argmax(dim=1)
        
        train_acc = (predict == y_batch).sum().item() / batch_size
        
        loss = criterion(scores, y_batch)
        train_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    train_loss /= batch_size
#     print('Train epoch : %s,  loss : %s,  accuracy :%.3f'%(epoch, train_loss / batch_size, train_acc))
    
    optimizer = adjust_learning_rate(optimizer, epoch, lr, 1) # adjust learning_rate while training
    
    model.eval()
    scores = model(x_test)
    predict = F.softmax(scores, dim=1).argmax(dim = 1)

    acc = (predict == y_test).sum().item() / y_test.size(0)
    loss = criterion(scores, y_test.long())

#     print('Test Epoch : %s, Test Loss : %.03f , Test Accuracy : %.03f'%(epoch, loss.item()/y_test.size(0), acc))
    print("Epoch:", epoch, train_loss, train_acc, loss,  acc)


LR is set to 8.7e-05
Epoch: 1 25.628804445266724 0.5625 tensor(192.2652, grad_fn=<NllLossBackward>) 0.567741935483871
LR is set to 7.569e-05
Epoch: 2 20.370041131973267 0.875 tensor(170.0465, grad_fn=<NllLossBackward>) 0.6193548387096774
LR is set to 6.58503e-05
Epoch: 3 15.623573184013367 0.625 tensor(144.1392, grad_fn=<NllLossBackward>) 0.7161290322580646
LR is set to 5.7289760999999995e-05
Epoch: 4 11.253332167863846 0.8125 tensor(125.8042, grad_fn=<NllLossBackward>) 0.7161290322580646
LR is set to 4.984209207e-05
Epoch: 5 8.51934415102005 0.875 tensor(119.5243, grad_fn=<NllLossBackward>) 0.6967741935483871
LR is set to 4.3362620100900007e-05
Epoch: 6 6.130872130393982 0.9375 tensor(109.9962, grad_fn=<NllLossBackward>) 0.7483870967741936
LR is set to 3.7725479487783e-05
Epoch: 7 4.594887092709541 1.0 tensor(109.4826, grad_fn=<NllLossBackward>) 0.7290322580645161
LR is set to 3.282116715437121e-05
Epoch: 8 3.541267551481724 0.9375 tensor(108.9555, grad_fn=<NllLossBackward>) 0.7290322